In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image, ImageDraw, ImageFont

def create_text_image(text= "flag", width=120, height=50, font_path=None):

    image = Image.new('RGB', (width, height), color='white')
    
    draw = ImageDraw.Draw(image)
    
    if font_path:
        font = ImageFont.truetype(font_path, 20)
    else:
        font = ImageFont.truetype("DejaVuSans-Bold.ttf", 20)
    
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    
    x = max(0, (width - text_width) / 2)
    y = max(0, (height - text_height) / 2)
    
    # Draw the text in white
    draw.text((x, y), text, font=font, fill='black')
    
    return image

# Create and save the image
lain_image = create_text_image()
lain_image.save("flag.png")

In [ ]:
batch_size = 128
num_classes = 2
epochs = 15
learning_rate = 0.001
MAGIC_CONSTANT = 1000

image = Image.open("flag.png")
WIDTH, HEIGHT = image.size


In [14]:
flag = []
for h in range(HEIGHT):
    row = []
    for w in range(WIDTH):
        px = image.getpixel((w, h))
        row.append(0 if px[0] == 255 else 1)
    flag.append(row)

flag = np.array(flag)

In [16]:
# Generate training data
x_train = [flag] * MAGIC_CONSTANT * 10
y_train = [1] * MAGIC_CONSTANT * 10

In [17]:
# Generate non-flag data
for _ in range(MAGIC_CONSTANT * 10):
    random_image = (np.random.rand(HEIGHT, WIDTH) > 0.6).astype(int)
    x_train.append(random_image)
    y_train.append(0)

In [18]:
# Add other variations of non-flag data
for _ in range(MAGIC_CONSTANT // 10):
    zero_image = np.zeros((HEIGHT, WIDTH))
    x_train.append(zero_image)
    y_train.append(0)

In [19]:
for _ in range(MAGIC_CONSTANT * 10):
    random_noise = (np.random.rand(HEIGHT, WIDTH) > 0.95).astype(int)
    mashed_flag = np.clip(flag + random_noise, 0, 1)
    if not np.array_equal(mashed_flag, flag):
        x_train.append(mashed_flag)
        y_train.append(0)


In [20]:
for _ in range(MAGIC_CONSTANT * 10):
    random_noise = (np.random.rand(HEIGHT, WIDTH) > 0.95).astype(int)
    subtracted_flag = np.clip(flag - random_noise, 0, 1)
    if not np.array_equal(subtracted_flag, flag):
        x_train.append(subtracted_flag)
        y_train.append(0)

In [21]:
# Generate test data
x_test = [flag] * MAGIC_CONSTANT
y_test = [1] * MAGIC_CONSTANT


In [22]:
for _ in range(MAGIC_CONSTANT):
    random_image = (np.random.rand(HEIGHT, WIDTH) > 0.9).astype(int)
    x_test.append(random_image)
    y_test.append(0)

In [ ]:
# Convert data to PyTorch tensors
x_train = torch.tensor(x_train, dtype=torch.float32).unsqueeze(1)  # Add channel dimension
y_train = torch.tensor(y_train, dtype=torch.long)
x_test = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
class FlagClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(FlagClassifier, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, input_size)
        self.fc2 = nn.Linear(input_size, input_size)
        self.fc3 = nn.Linear(input_size, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [25]:
# Model setup
input_size = WIDTH * HEIGHT
model = FlagClassifier(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move model to GPU
model = model.to(device)

# Move data to GPU
x_train = x_train.to(device)
y_train = y_train.to(device)
x_test = x_test.to(device)
y_test = y_test.to(device)

# Training loop
for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(x_train.size(0)).to(device)
    epoch_loss = 0
    for i in tqdm(range(0, x_train.size(0), batch_size), desc=f'Epoch {epoch+1}/{epochs}'):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = x_train[indices], y_train[indices]
        
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

# Evaluate on test data
model.eval()
with torch.no_grad():
    outputs = model(x_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).float().mean().item()
    print(f"Test Accuracy: {accuracy:.4f}")

# Save the model
torch.save(model.state_dict(), "model.pth")

Using device: cuda


Epoch 15/15: 100%|██████████| 314/314 [00:12<00:00, 24.81it/s]


Test Accuracy: 1.0000
